### Import packages

In [1]:
import sys
import math
import numpy as np
import pandas as pd

### Define genome

In [184]:
genome = "AGPv4"
genDir = "".join(["/scratch/Shares/dowell/genomes/",genome,"/"])

### Import file

In [6]:
file1 = "".join([genDir,genome,"_refseq.gtf"])

regions = pd.read_table(file1,header=None,index_col=False,names=["chrom","info","ann_type","coor1","coor2","dot","strand","dot2","gene_info"])

/home/lsanford/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Separate out relevant data and initialize bedfile dataframe

In [31]:
all_transcripts = regions["gene_info"].str.split("transcript_id \"", expand = True)
all_transcripts = all_transcripts[1].str.split("\"", expand = True)
regions["transcript"] = all_transcripts[0]
exons = regions[regions["ann_type"] == "exon"]
startstop = regions[regions["ann_type"].str.contains('codon')]
transcripts = all_transcripts[0].unique()

bedlines = pd.DataFrame(columns=["chrom","coor1","coor2","transcript",
                                 "zero1","strand","three","five","zero2",
                                 "num_ex","length","starts"])

### Loop through transcripts and extract coordinates

In [32]:
for ts in range(len(transcripts)):
    curr_ex = exons[exons["transcript"] == transcripts[ts]]
    
    if curr_ex.empty:
        continue
    else:
        mincoor = min(curr_ex["coor1"]) - 1
        maxcoor = max(curr_ex["coor2"]) + 1
        numex = len(curr_ex)
        strand = curr_ex.iloc[0,6]
        chrom = curr_ex.iloc[0,0]

        curr_ex["length"] = curr_ex["coor2"] - curr_ex["coor1"] + 1
        curr_ex["starts"] = curr_ex["coor1"] - mincoor - 1
        bedlen = ",".join(map(str,list(curr_ex["length"])))
        bedlen = "".join([bedlen,","])
        bedst = ",".join(map(str,list(curr_ex["starts"])))
        bedst = "".join([bedst,","])

        curr_st = startstop[startstop["transcript"] == transcripts[ts]]
        
        if curr_st.empty:
            three = mincoor
            five = maxcoor
        else:
            three = min(curr_st["coor1"]) - 1
            five = max(curr_st["coor2"]) + 1

        newline = pd.DataFrame([[chrom,mincoor,maxcoor,transcripts[ts],0,strand,
                                three,five,0,numex,bedlen,bedst]],
                               columns=["chrom","coor1","coor2","transcript",
                                        "zero1","strand","three","five","zero2",
                                        "num_ex","length","starts"])

        bedlines = bedlines.append(newline)

<ipython-input-32-52ec42403bf6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_ex["length"] = curr_ex["coor2"] - curr_ex["coor1"] + 1
<ipython-input-32-52ec42403bf6>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_ex["starts"] = curr_ex["coor1"] - mincoor - 1


### Write out file

In [202]:
outfile = ''.join([genDir,'/',genome,'_refseq.bed'])
bedlines.to_csv(outfile,sep="\t",index=False,index_label=False,header=False)